# AI Tour Planner: Smart Tourism Research Component

## Overview
This notebook implements the **Intelligent Multi-Objective Tour Planner** for the Smart Tourism system.
It uses a **Genetic Algorithm (GA)** to optimize a user's route through Kandy, Sri Lanka, balancing:
1.  **Distance** (Real road network data from OpenStreetMap)
2.  **Weather Risk** (From Component 3)
3.  **Crowd Levels** (From Component 1)

## 1. Setup & Imports
Install necessary libraries before running.

In [ ]:
!pip install osmnx networkx folium pandas matplotlib scikit-learn

In [ ]:
import osmnx as ox
import networkx as nx
import folium
import random
import numpy as np
from itertools import permutations

# Configure OSMnx
ox.settings.use_cache = True
ox.settings.log_console = True

## 2. Data Acquisition
We fetch the drivable street network for **Kandy, Sri Lanka** and define our key tourist attractions.

In [ ]:
LOCATION_NAME = "Kandy, Sri Lanka"
print(f"Fetching map data for {LOCATION_NAME}...")

# Fetch Graph
G = ox.graph_from_place(LOCATION_NAME, network_type='drive')

# Define Key Attractions (Kandy)
attractions = {
    "Temple of the Tooth": (7.2936, 80.6413),
    "Kandy Lake": (7.2931, 80.6430),
    "Peradeniya Gardens": (7.2687, 80.5966),
    "Bahirawakanda Buddha": (7.2995, 80.6318),
    "Ceylon Tea Museum": (7.2689, 80.6300),
    "Udawatta Kele": (7.2990, 80.6445)
}

## 3. Distance Matrix Calculation
We calculate the real-world shortest path distance between every pair of attractions using the OSM street network.

In [ ]:
# Find nearest network nodes for each attraction
nodes = {}
for name, (lat, lon) in attractions.items():
    nodes[name] = ox.distance.nearest_nodes(G, lon, lat)

locations_list = list(attractions.keys())
n_locs = len(locations_list)
dist_matrix = np.zeros((n_locs, n_locs))

locations_list = list(attractions.keys())
n_locs = len(locations_list)
dist_matrix = np.zeros((n_locs, n_locs))

print("Calculating Distance Matrix (this may take a moment)...")
for i in range(n_locs):
    for j in range(n_locs):
        if i != j:
            u = nodes[locations_list[i]]
            v = nodes[locations_list[j]]
            try:
                dist = nx.shortest_path_length(G, u, v, weight='length')
                dist_matrix[i][j] = dist
            except nx.NetworkXNoPath:
                dist_matrix[i][j] = 999999 

print("Distance Matrix Ready.")

## 4. Genetic Algorithm (The AI Planner)
We implement a GA to find the optimal sequence of visits that minimizes cost (Distance + Penalties).

### Novelty: Multi-Objective Fitness Function
$$Fitness = \frac{1}{TotalDist + RainPenalty + CrowdPenalty}$$

In [ ]:
def calculate_fitness(route, dist_matrix, weather_risks, crowd_levels):
    total_dist = 0
    penalty = 0
    
    for i in range(len(route) - 1):
        u_idx = route[i]
        v_idx = route[i+1]
        total_dist += dist_matrix[u_idx][v_idx]
        
        # Research Novelty: Context-Aware Penalties
        # Example: Heavy Rain at Outdoor Location
        if locations_list[v_idx] == "Peradeniya Gardens" and weather_risks[i] > 0.7:
            penalty += 5000 
            
        # Example: Peak Crowds at Major Temple
        if locations_list[v_idx] == "Temple of the Tooth" and crowd_levels[i] == "High":
            penalty += 3000
            
    return 1 / (total_dist + penalty + 1e-6)

# Mock Data simulating real-time API inputs
mock_weather_risk = [0.2, 0.8, 0.1, 0.3, 0.4, 0.1]
mock_crowd_level = ["Low", "High", "Medium", "Low", "Low", "Medium"]

# GA Parameters
POP_SIZE = 50
GENERATIONS = 100
MUTATION_RATE = 0.2

# Run Evolution
population = []
base_indices = list(range(n_locs))
for _ in range(POP_SIZE):
    population.append(random.sample(base_indices, n_locs))

for gen in range(GENERATIONS):
    fitness_scores = [calculate_fitness(ind, dist_matrix, mock_weather_risk, mock_crowd_level) for ind in population]
    sorted_pop = [x for _, x in sorted(zip(fitness_scores, population), key=lambda pair: pair[0], reverse=True)]
    new_pop = sorted_pop[:2] # Elitism
    
    while len(new_pop) < POP_SIZE:
        parent1 = random.choice(sorted_pop[:10])
        parent2 = random.choice(sorted_pop[:10])
        child = parent1[:]
        if random.random() < MUTATION_RATE:
            idx1, idx2 = random.sample(range(n_locs), 2)
            child[idx1], child[idx2] = child[idx2], child[idx1]
        new_pop.append(child)
    population = new_pop

best_route_indices = population[0]
best_route_names = [locations_list[i] for i in best_route_indices]

print("Best Route Found by AI:", best_route_names)

## 5. Visualization
Visualizing the optimized route on an interactive map.

In [ ]:
m = folium.Map(location=attractions["Kandy Lake"], zoom_start=13)

for name, (lat, lon) in attractions.items():
    color = 'red' if name == "Temple of the Tooth" else 'blue'
    folium.Marker([lat, lon], popup=name, icon=folium.Icon(color=color)).add_to(m)

for i in range(len(best_route_indices) - 1):
    u = nodes[locations_list[best_route_indices[i]]]
    v = nodes[locations_list[best_route_indices[i+1]]]
    try:
        path_nodes = nx.shortest_path(G, u, v, weight='length')
        path_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in path_nodes]
        folium.PolyLine(path_coords, color="green", weight=4, opacity=0.8).add_to(m)
    except:
        pass

m